In [8]:
getwd()
if ( is.null(environment()$this_notebook_dir) ) {
    this_notebook_dir <- getwd()
    setwd(paste0(getwd(), '/..'))
}
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [9]:
source('helpers.R')
source('validation_utils.R')

kaggle.house.loadLibraries()
data <- kaggle.house.loadData()

training <- data$train
testing <- data$test

c(dim(training), dim(testing))

training <- training %>% mutate(sale_price_log = log(SalePrice))

[1] 1460   80 1459   79

In [10]:
char_vars_names <- training %>% purrr::map(~is.character(.)) %>% purrr::keep(~.) %>% names %>% sort
char_vars_names

[1] "Alley"         "BldgType"      "BsmtCond"      "BsmtExposure" 
 [5] "BsmtFinType1"  "BsmtFinType2"  "BsmtQual"      "CentralAir"   
 [9] "Condition1"    "Condition2"    "Electrical"    "ExterCond"    
[13] "Exterior1st"   "Exterior2nd"   "ExterQual"     "Fence"        
[17] "FireplaceQu"   "Foundation"    "Functional"    "GarageCond"   
[21] "GarageFinish"  "GarageQual"    "GarageType"    "Heating"      
[25] "HeatingQC"     "HouseStyle"    "KitchenQual"   "LandContour"  
[29] "LandSlope"     "LotConfig"     "LotShape"      "MasVnrType"   
[33] "MiscFeature"   "MSZoning"      "Neighborhood"  "PavedDrive"   
[37] "PoolQC"        "RoofMatl"      "RoofStyle"     "SaleCondition"
[41] "SaleType"      "Street"        "Utilities"

In [11]:
removify <- function(trans, attr_name) {
    function(df, remove=T) {
        df <- trans(df)
        if ( remove ) {
            df[, attr_name] <- NULL
        }
        df
    }
}

In [12]:
transform <- list()

In [13]:
transform$Alley <- removify(
    function (df) {
        df %>% mutate(has_alley_access = ifelse(!is.na(Alley), 1, 0))
    }, 
    "Alley")

training %>% group_by(Alley) %>% count
tranformed <- transform$Alley(training, remove=F)
tranformed %>% group_by(has_alley_access, Alley) %>% count

Alley,n
Grvl,50
Pave,41
NA,1369


has_alley_access,Alley,n
0,NA,1369
1,Grvl,50
1,Pave,41


In [14]:
transform$CentralAir <- removify(
    function (df) {
        df %>% mutate(has_central_air = ifelse(CentralAir == 'Y', 1, 0))
    }, 
    "CentralAir")

training %>% group_by(CentralAir) %>% count
tranformed <- transform$CentralAir(training, remove=F)
tranformed %>% group_by(has_central_air, CentralAir) %>% count

CentralAir,n
N,95
Y,1365


has_central_air,CentralAir,n
0,N,95
1,Y,1365


In [88]:
transform$Electrical <- removify(
    function (df) {
        df %>% mutate(standard_electrical = ifelse(Electrical == 'SBrkr' | is.na(Electrical), 1, 0))
    }, 
    "Electrical")

training %>% group_by(Electrical) %>% count
tranformed <- transform$Electrical(training, remove=F)
tranformed %>% group_by(standard_electrical, Electrical) %>% count

Electrical,n
FuseA,94
FuseF,27
FuseP,3
Mix,1
SBrkr,1334
NA,1


standard_electrical,Electrical,n
0,FuseA,94
0,FuseF,27
0,FuseP,3
0,Mix,1
1,SBrkr,1334
1,NA,1


In [15]:
transform$Heating <- removify(
    function (df) {
        df %>% mutate(heating_air_furnace = ifelse(Heating == 'GasA', 1, 0))
    }, 
    "Heating")

training %>% group_by(Heating) %>% count
tranformed <- transform$Heating(training, remove=F)
tranformed %>% group_by(heating_air_furnace, Heating) %>% count

Heating,n
Floor,1
GasA,1428
GasW,18
Grav,7
OthW,2
Wall,4


heating_air_furnace,Heating,n
0,Floor,1
0,GasW,18
0,Grav,7
0,OthW,2
0,Wall,4
1,GasA,1428


In [16]:
transform$LandContour <- removify(
    function (df) {
        df %>% mutate(is_land_level = ifelse(LandContour == 'Lvl', 1, 0))
    }, 
    "LandContour")

training %>% group_by(LandContour) %>% count
tranformed <- transform$LandContour(training, remove=F)
tranformed %>% group_by(is_land_level, LandContour) %>% count

LandContour,n
Bnk,63
HLS,50
Low,36
Lvl,1311


is_land_level,LandContour,n
0,Bnk,63
0,HLS,50
0,Low,36
1,Lvl,1311


In [17]:
transform$LotShape <- removify(
    function (df) {
        df %>% mutate(is_lotshape_regular = ifelse(LotShape == 'Reg', 1, 0))
    }, 
    "LotShape")

training %>% group_by(LotShape) %>% count
tranformed <- transform$LotShape(training, remove=F)
tranformed %>% group_by(is_lotshape_regular, LotShape) %>% count

LotShape,n
IR1,484
IR2,41
IR3,10
Reg,925


is_lotshape_regular,LotShape,n
0,IR1,484
0,IR2,41
0,IR3,10
1,Reg,925


In [18]:
transform$MiscFeature <- removify(
    function (df) {
        df %>% mutate(has_misc_feature = ifelse(!is.na(MiscFeature), 1, 0))
    }, 
    "MiscFeature")

training %>% group_by(MiscFeature) %>% count
tranformed <- transform$MiscFeature(training, remove=F)
tranformed %>% group_by(has_misc_feature, MiscFeature) %>% count

MiscFeature,n
Gar2,2
Othr,2
Shed,49
TenC,1
NA,1406


has_misc_feature,MiscFeature,n
0,NA,1406
1,Gar2,2
1,Othr,2
1,Shed,49
1,TenC,1


In [19]:
transform$PavedDrive <- removify(
    function (df) {
        df %>% mutate(has_paved_drive = ifelse(PavedDrive == 'Y', 1, 0))
    }, 
    "PavedDrive")

training %>% group_by(PavedDrive) %>% count
tranformed <- transform$PavedDrive(training, remove=F)
tranformed %>% group_by(has_paved_drive, PavedDrive) %>% count

PavedDrive,n
N,90
P,30
Y,1340


has_paved_drive,PavedDrive,n
0,N,90
0,P,30
1,Y,1340


In [20]:
transform$PoolQC <- removify(
    function (df) {
        df %>% mutate(has_pool = ifelse(!is.na(PoolQC), 1, 0))
    }, 
    "PoolQC")

training %>% group_by(PoolQC) %>% count
tranformed <- transform$PoolQC(training, remove=F)
tranformed %>% group_by(has_pool, PoolQC) %>% count

PoolQC,n
Ex,2
Fa,2
Gd,3
NA,1453


has_pool,PoolQC,n
0,NA,1453
1,Ex,2
1,Fa,2
1,Gd,3


In [21]:
transform$RoofMatl <- removify(
    function (df) {
        df %>% mutate(standard_roof_material = ifelse(RoofMatl == 'CompShg', 1, 0))
    }, 
    "RoofMatl")

training %>% group_by(RoofMatl) %>% count
tranformed <- transform$RoofMatl(training, remove=F)
tranformed %>% group_by(standard_roof_material, RoofMatl) %>% count

RoofMatl,n
ClyTile,1
CompShg,1434
Membran,1
Metal,1
Roll,1
Tar&Grv,11
WdShake,5
WdShngl,6


standard_roof_material,RoofMatl,n
0,ClyTile,1
0,Membran,1
0,Metal,1
0,Roll,1
0,Tar&Grv,11
0,WdShake,5
0,WdShngl,6
1,CompShg,1434


In [22]:
transform$Street <- removify(
    function (df) {
        df %>% mutate(is_street_paved = ifelse(Street == 'Pave', 1, 0))
    }, 
    "Street")

training %>% group_by(Street) %>% count
tranformed <- transform$Street(training, remove=F)
tranformed %>% group_by(is_street_paved, Street) %>% count

Street,n
Grvl,6
Pave,1454


is_street_paved,Street,n
0,Grvl,6
1,Pave,1454


In [57]:
xxx <- function (df, attr_name, new_attr_name) {
    attr_name <- enquo(attr_name)
    df %>% group_by(!!attr_name) %>% mutate(!!new_attr_name := median(sale_price_log))
}

df <- xxx(training, BldgType, "building_type")
df %>% group_by(BldgType) %>% summarise(min(building_type), max(building_type), median(sale_price_log))

BldgType,min(building_type),max(building_type),median(sale_price_log)
1Fam,12.03112,12.03112,12.03112
2fmCon,11.75587,11.75587,11.75587
Duplex,11.82026,11.82026,11.82026
Twnhs,11.83138,11.83138,11.83138
TwnhsE,12.05641,12.05641,12.05641


In [60]:
group_averaging_tran <- function (attr_name, new_attr_name) {
    
    attr_name <- enquo(attr_name)
    
    function (df, remove=T) {
        df.new <- df %>%
            group_by(!!attr_name) %>%
            mutate(!!new_attr_name := median(sale_price_log))
        if ( remove ) {
            df.new[, attr_name] <- NULL
        }
        df.new
    }
}

In [61]:
transform$BldgType <- group_averaging_tran(BldgType, "building_type")

training %>% group_by(BldgType) %>% count
tranformed <- transform$BldgType(training, remove=F)

tranformed %>% group_by(BldgType) %>% 
summarise(min(building_type), max(building_type), median(sale_price_log))

BldgType,n
1Fam,1220
2fmCon,31
Duplex,52
Twnhs,43
TwnhsE,114


BldgType,min(building_type),max(building_type),median(sale_price_log)
1Fam,12.03112,12.03112,12.03112
2fmCon,11.75587,11.75587,11.75587
Duplex,11.82026,11.82026,11.82026
Twnhs,11.83138,11.83138,11.83138
TwnhsE,12.05641,12.05641,12.05641


In [63]:
transform$BsmtCond <- group_averaging_tran(BsmtCond, "basement_condition")

training %>% group_by(BsmtCond) %>% count
tranformed <- transform$BsmtCond(training, remove=F)

tranformed %>% group_by(BsmtCond) %>% 
summarise(min(basement_condition), max(basement_condition), median(sale_price_log))

BsmtCond,n
Fa,45
Gd,65
Po,2
TA,1311
NA,37


BsmtCond,min(basement_condition),max(basement_condition),median(sale_price_log)
Fa,11.68267,11.68267,11.68267
Gd,12.17499,12.17499,12.17499
Po,11.06554,11.06554,11.06554
TA,12.01370,12.01370,12.01370
NA,11.53077,11.53077,11.53077


In [65]:
transform$BsmtExposure <- group_averaging_tran(BsmtExposure, "basement_exposure")

training %>% group_by(BsmtExposure) %>% count
tranformed <- transform$BsmtExposure(training, remove=F)

tranformed %>% group_by(BsmtExposure) %>% 
summarise(min(basement_exposure), max(basement_exposure), median(sale_price_log))

BsmtExposure,n
Av,221
Gd,134
Mn,114
No,953
NA,38


BsmtExposure,min(basement_exposure),max(basement_exposure),median(sale_price_log)
Av,12.13270,12.13270,12.13270
Gd,12.33256,12.33256,12.33256
Mn,12.11413,12.11413,12.11413
No,11.94471,11.94471,11.94471
NA,11.55216,11.55216,11.55216


In [68]:
transform$BsmtFinType1 <- group_averaging_tran(BsmtFinType1, "basement_finish1")

training %>% group_by(BsmtFinType1) %>% count
tranformed <- transform$BsmtFinType1(training, remove=F)

tranformed %>% group_by(BsmtFinType1) %>% 
summarise(min(basement_finish1), max(basement_finish1), median(sale_price_log))

BsmtFinType1,n
ALQ,220
BLQ,148
GLQ,418
LwQ,74
Rec,133
Unf,430
NA,37


BsmtFinType1,min(basement_finish1),max(basement_finish1),median(sale_price_log)
ALQ,11.91338,11.91338,11.91338
BLQ,11.84295,11.84295,11.84295
GLQ,12.27256,12.27256,12.27256
LwQ,11.84223,11.84223,11.84223
Rec,11.86358,11.86358,11.86358
Unf,11.99381,11.99381,11.99381
NA,11.53077,11.53077,11.53077


In [71]:
transform$BsmtFinType2 <- group_averaging_tran(BsmtFinType2, "basement_finish2")

training %>% group_by(BsmtFinType2) %>% count
tranformed <- transform$BsmtFinType2(training, remove=F)

tranformed %>% group_by(BsmtFinType2) %>% 
summarise(min(basement_finish2), max(basement_finish2), median(sale_price_log))

BsmtFinType2,n
ALQ,19
BLQ,33
GLQ,14
LwQ,46
Rec,54
Unf,1256
NA,38


BsmtFinType2,min(basement_finish2),max(basement_finish2),median(sale_price_log)
ALQ,12.07197,12.07197,12.07197
BLQ,11.87060,11.87060,11.87060
GLQ,12.22033,12.22033,12.22033
LwQ,11.94469,11.94469,11.94469
Rec,11.90995,11.90995,11.90995
Unf,12.02575,12.02575,12.02575
NA,11.55216,11.55216,11.55216


In [73]:
transform$BsmtQual <- group_averaging_tran(BsmtQual, "basement_height_quality")

training %>% group_by(BsmtQual) %>% count
tranformed <- transform$BsmtQual(training, remove=F)

tranformed %>% group_by(BsmtQual) %>% 
summarise(min(basement_height_quality), max(basement_height_quality), median(sale_price_log))

BsmtQual,n
Ex,121
Fa,35
Gd,618
TA,649
NA,37


BsmtQual,min(basement_height_quality),max(basement_height_quality),median(sale_price_log)
Ex,12.66981,12.66981,12.66981
Fa,11.62625,11.62625,11.62625
Gd,12.16562,12.16562,12.16562
TA,11.81673,11.81673,11.81673
NA,11.53077,11.53077,11.53077


In [83]:
transform$Condition <- function (df) {
    training %>% mutate(
        adjacent_railroad = ifelse(Condition1 %in% c('RRAe', 'RRAn', 'RRNe', 'RRNn') | 
                                   Condition2 %in% c('RRAe', 'RRAn', 'RRNe', 'RRNn'),
                                   1, 0),

        adjacent_traffic_street = ifelse(Condition1 %in% c('Artery', 'Feedr') | 
                                   Condition2 %in% c('Artery', 'Feedr'),
                                   1, 0),

        adjacent_positive = ifelse(Condition1 %in% c('PosA', 'PosN') | 
                                   Condition2 %in% c('PosA', 'PosN'),
                                   1, 0)
    )
}

tranformed <- transform$Condition(training)

training %>% group_by(Condition1) %>% summarise(avg = median(sale_price_log), n=n())
training %>% group_by(Condition2) %>% summarise(avg = median(sale_price_log), n=n())

tranformed %>% 
group_by(adjacent_railroad, adjacent_traffic_street, adjacent_positive, Condition1, Condition2) %>% 
count

Condition1,avg,n
Artery,11.69149,48
Feedr,11.84940,81
Norm,12.02275,1260
PosA,12.26106,8
PosN,12.20607,19
RRAe,11.86710,11
RRAn,12.05227,26
RRNe,12.15853,2
RRNn,12.27373,5


Condition2,avg,n
Artery,11.57004,2
Feedr,11.75586,6
Norm,12.00457,1445
PosA,12.69158,1
PosN,12.49388,2
RRAe,12.15478,1
RRAn,11.82704,1
RRNn,11.43533,2


adjacent_railroad,adjacent_traffic_street,adjacent_positive,Condition1,Condition2,n
0,0,0,Norm,Norm,1260
0,0,1,PosA,Norm,8
0,0,1,PosN,Norm,17
0,0,1,PosN,PosN,2
0,1,0,Artery,Artery,2
0,1,0,Artery,Norm,45
0,1,0,Feedr,Feedr,1
0,1,0,Feedr,Norm,76
0,1,1,Artery,PosA,1
1,0,0,RRAe,Norm,11


In [89]:
setdiff(char_vars_names, transform %>% names) %>% sort

[1] "Condition1"    "Condition2"    "ExterCond"     "Exterior1st"  
 [5] "Exterior2nd"   "ExterQual"     "Fence"         "FireplaceQu"  
 [9] "Foundation"    "Functional"    "GarageCond"    "GarageFinish" 
[13] "GarageQual"    "GarageType"    "HeatingQC"     "HouseStyle"   
[17] "KitchenQual"   "LandSlope"     "LotConfig"     "MasVnrType"   
[21] "MSZoning"      "Neighborhood"  "RoofStyle"     "SaleCondition"
[25] "SaleType"      "Utilities"

In [105]:
training %>% group_by(ExterQual, ExterCond) %>% summarise(avg = median(sale_price_log), n=n())

transform$ExterQualCond <- function (df) {
    training %>% mutate(
        exterior_qual_cond = case_when(
            
            ExterQual == 'Ex' & ExterCond %in% c('Ex', 'Gd', 'TA') ~ 'Ex',
            ExterQual == 'Ex' & ExterCond == 'Fa' ~ 'good',
            
            ExterQual == 'Gd' & ExterCond %in% c('Ex', 'Gd', 'TA') ~ 'Gd',
            ExterQual == 'Gd' & ExterCond == 'Fa' ~ 'typical',
            
            ExterQual == 'TA' & ExterCond != 'Po' ~ 'TA',
            ExterQual == 'Fa' & ExterCond != 'Po' ~ 'Fa',
            T ~ 'Po'
        )
    )
}

tranformed <- transform$ExterQualCond(training)
tranformed %>% group_by(exterior_qual_cond) %>% summarise(avg = median(sale_price_log), n=n())

ExterQual,ExterCond,avg,n
Ex,Ex,12.69158,1
Ex,Gd,12.49313,3
Ex,TA,12.83248,48
Fa,Fa,11.01036,6
Fa,TA,11.50104,8
Gd,Gd,12.25486,35
Gd,TA,12.30592,453
TA,Ex,11.83380,2
TA,Fa,11.68462,22
TA,Gd,11.84940,108


exterior_qual_cond,avg,n
Ex,12.80655,52
Fa,11.31751,14
Gd,12.30138,488
Po,11.24505,1
TA,11.84582,905
